In [66]:
def check_nan(df):
    cols = df.columns
    for col in cols:
        print(col+'column:')
        print(df[col].isnull().value_counts())
        print("*********************************")

# Theory

https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-for-ensemble-models/

https://medium.com/analytics-vidhya/an-end-to-end-guide-to-understand-the-math-behind-xgboost-72c07acb4afb

https://www.kaggle.com/dansbecker/xgboost

https://towardsdatascience.com/fine-tuning-xgboost-in-python-like-a-boss-b4543ed8b1e

XGBoost is an implementation of the Gradient Boosted Decision Trees algorithm.<br>
XGBoost use Bagging and Boosting. When we fit both these models, they would yield different results. Decision trees are said to be associated with high variance due to this behavior. Bagging or boosting aggregation helps to reduce the variance in any learner. Several decision trees which are generated in parallel, form the base learners of bagging technique. Data sampled with replacement is fed to these learners for training. The final prediction is the averaged output from all the learners.<br>
In boosting, the trees are built sequentially such that each subsequent tree aims to reduce the errors of the previous tree. Each tree learns from its predecessors and updates the residual errors. Hence, the tree that grows next in the sequence will learn from an updated version of the residuals.<br>
<mark>Having a large number of trees might lead to overfitting. So, it is necessary to carefully choose the stopping criteria for boosting.</mark>

# Model

### Ready Data

In [81]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer


df  = pd.read_csv('data/Housing Prices Competition for Kaggle Learn Users/melb_data.csv',na_values=["NA","?"])
df.head(2)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0


In [67]:
check_nan(df)

Suburbcolumn:
False    13580
Name: Suburb, dtype: int64
*********************************
Addresscolumn:
False    13580
Name: Address, dtype: int64
*********************************
Roomscolumn:
False    13580
Name: Rooms, dtype: int64
*********************************
Typecolumn:
False    13580
Name: Type, dtype: int64
*********************************
Pricecolumn:
False    13580
Name: Price, dtype: int64
*********************************
Methodcolumn:
False    13580
Name: Method, dtype: int64
*********************************
SellerGcolumn:
False    13580
Name: SellerG, dtype: int64
*********************************
Datecolumn:
False    13580
Name: Date, dtype: int64
*********************************
Distancecolumn:
False    13580
Name: Distance, dtype: int64
*********************************
Postcodecolumn:
False    13580
Name: Postcode, dtype: int64
*********************************
Bedroom2column:
False    13580
Name: Bedroom2, dtype: int64
*********************************
Bathro

In [89]:
df = df.dropna(axis=0)

In [90]:
melbourne= ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude','Price']
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
df = df[melbourne]

In [91]:
df.head(2)

,Rooms,Bathroom,Landsize,Lattitude,Longtitude,Price
1,2,1.0,156.0,-37.8079,144.9934,1035000.0
2,3,2.0,134.0,-37.8093,144.9944,1465000.0


In [92]:
x,y= df[melbourne_features],df.Price


In [96]:
x.head()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
1,2,1.0,156.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,144.9944
4,4,1.0,120.0,-37.8072,144.9941
6,3,2.0,245.0,-37.8024,144.9993
7,2,1.0,256.0,-37.8060,144.9954


In [94]:
x.as_matrix()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[ 2.0000000e+00,  1.0000000e+00,  1.5600000e+02, -3.7807900e+01,
         1.4499340e+02],
       [ 3.0000000e+00,  2.0000000e+00,  1.3400000e+02, -3.7809300e+01,
         1.4499440e+02],
       [ 4.0000000e+00,  1.0000000e+00,  1.2000000e+02, -3.7807200e+01,
         1.4499410e+02],
       ...,
       [ 1.0000000e+00,  1.0000000e+00,  0.0000000e+00, -3.7855880e+01,
         1.4489936e+02],
       [ 2.0000000e+00,  1.0000000e+00,  0.0000000e+00, -3.7855810e+01,
         1.4499025e+02],
       [ 6.0000000e+00,  3.0000000e+00,  1.0870000e+03, -3.7810380e+01,
         1.4489389e+02]])

In [97]:
train_X, test_X, train_y, test_y = train_test_split(x.as_matrix(), y.as_matrix(), test_size=0.25)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [98]:
my_imputer = Imputer() # Imputation transformer for completing missing values.
# strategy : string, optional (default=”mean”)
train_X = my_imputer.fit_transform(train_X) 
test_X = my_imputer.transform(test_X)

### describe model and feed it

In [117]:
from xgboost import XGBRegressor

my_model = XGBRegressor()

eval_set = [(train_X, train_y), (test_X, test_y )]
eval_metric = ["auc","error"]
# Add silent=True to avoid printing out updates with each cycle
%time my_model.fit(train_X, train_y, eval_metric=eval_metric,  verbose=True)


CPU times: user 248 ms, sys: 4.06 ms, total: 252 ms
Wall time: 253 ms


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [102]:
#!pip install xgboost

In [103]:
predictions = my_model.predict(test_X)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))

Mean Absolute Error : 223725.01730451098


# XGBoost  parameters

<b>1. n_estimators:</b> <br>
specifies how many times to go through the modeling cycle .<br>
In the underfitting vs overfitting graph, n_estimators moves you further to the right. Too low a value causes underfitting, which is inaccurate predictions on both training data and new data. Too large a value causes overfitting, which is accurate predictions on training data, but inaccurate predictions on new data (which is what we care about). You can experiment with your dataset to find the ideal. Typical values range from 100-1000, though this depends a lot on the learning rate discussed below.

<b>2. early_stopping_rounds: </b> <br>
The argument early_stopping_rounds offers a way to automatically find the ideal value. <br>
Early stopping causes the model to stop iterating when the validation score stops improving, even if we aren't at the hard stop for n_estimators. It's smart to set a high value for n_estimators and then use early_stopping_rounds to find the optimal time to stop iterating.<br>
Since random chance sometimes causes a single round where validation scores don't improve, you need to specify a number for how many rounds of straight deterioration to allow before stopping. early_stopping_rounds = 5 is a reasonable value. Thus we stop after 5 straight rounds of deteriorating validation scores.

<b>(you can use a higher value of n_estimators without overfitting. If you use early stopping, the appropriate number of trees will be set automatically.)</b>
                                                                                   

<b>3. learning_rate:</b><br>
Instead of getting predictions by simply adding up the predictions from each component model, we will multiply the predictions from each model by a small number before adding them in. This means each tree we add to the ensemble helps us less. In practice, this reduces the model's propensity to overfit.

(a small learning rate (and large number of estimators) will yield more accurate XGBoost models, though it will also take the model longer to train since it does more iterations through the cycle.)

<b>n_jobs:</b><br>
On larger datasets where runtime is a consideration, you can use parallelism to build your models faster. It's common to set the parameter n_jobs equal to the number of cores on your machine. On smaller datasets, this won't help.

In [104]:
my_model = XGBRegressor(n_estimators=1000)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [105]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

Fill reasonable values for key inputs:

learning_rate: 0.01

n_estimators: 100 if the size of your data is high, 1000 is if it is medium-low

max_depth: 3

subsample: 0.8

colsample_bytree: 1

gamma: 1

Run model.fit(eval_set, eval_metric) and diagnose your first run, specifically the n_estimators parameter
Optimize max_depth parameter. It represents the depth of each tree, which is the maximum number of different features used in each tree. I recommend going from a low max_depth (3 for instance) and then increasing it incrementally by 1, and stopping when there’s no performance gain of increasing it. This will help simplify your model and avoid overfitting

Now play around with the learning rate and the features that avoids overfitting:

learning_rate: usually between 0.1 and 0.01. If you’re focused on performance and have time in front of you, decrease incrementally the learning rate while increasing the number of trees.

subsample, which is for each tree the % of rows taken to build the tree. I recommend not taking out too many rows, as performance will drop a lot. Take values from 0.8 to 1.

colsample_bytree: number of columns used by each tree. In order to avoid some columns to take too much credit for the prediction (think of it like in recommender systems when you recommend the most purchased products and forget about the long tail), take out a good proportion of columns. Values from 0.3 to 0.8 if you have many columns (especially if you did one-hot encoding), or 0.8 to 1 if you only have a few columns.

gamma: usually misunderstood parameter, it acts as a regularization parameter. Either 0, 1 or 5.
